### Accompanying notebook for Habitable Planets selection problem (student version; to fill)

In [ ]:
#Author: Viviana Acquaviva

#License: BSD but really should be TBD - just be nice.

In [ ]:
import pandas as pd
import numpy as np
import sklearn.tree
from sklearn.model_selection import train_test_split
from sklearn import datasets
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn import metrics 
from scipy import stats
from sklearn.model_selection import cross_val_predict, cross_val_score, cross_validate
from sklearn.model_selection import KFold, StratifiedKFold
from sklearn.model_selection import GridSearchCV

from sklearn import neighbors

import matplotlib.pyplot as plt
import matplotlib.patches as mpatches

In [ ]:
from io import StringIO  
from IPython.display import Image  
import pydotplus
from sklearn.tree import export_graphviz

In [ ]:
import matplotlib
font = {'size'   : 20}

matplotlib.rc('font', **font)
matplotlib.rc('xtick', labelsize=20) 
matplotlib.rc('ytick', labelsize=20) 
matplotlib.rcParams['figure.dpi'] = 300

In [ ]:
#Data from http://phl.upr.edu/projects/habitable-exoplanets-catalog/data/database

In [ ]:
df = pd.read_csv('phl_exoplanet_catalog.csv', sep = ',')

In [ ]:
df.columns

In [ ]:
df.describe()

In [ ]:
df.groupby('P_HABITABLE').count()

#### Start by lumping together Probably and Possibly Habitable

In [ ]:
bindf = df.drop('P_HABITABLE', axis = 1) #This has the binary classification

In [ ]:
bindf['P_HABITABLE'] = (np.logical_or((df.P_HABITABLE == 1) , (df.P_HABITABLE == 2))) #turn into binary

bindf['P_HABITABLE'] = bindf['P_HABITABLE'].astype(int)

In [ ]:
bindf.head()

### Let's select some columns.

S_MAG - star magnitude 

S_DISTANCE - star distance (parsecs)

S_METALLICITY - star metallicity (dex)

S_MASS - star mass (solar units)

S_RADIUS - star radius (solar units)

S_AGE - star age (Gy)

S_TEMPERATURE - star effective temperature (K)

S_LOG_G - star log(g)

P_DISTANCE - planet mean distance from the star (AU) 

P_FLUX - planet mean stellar flux (earth units)

P_PERIOD - planet period (days) 

### Going with the same features as Chapter 2.

In [ ]:
final_features = bindf[['S_MASS', 'P_PERIOD', 'P_DISTANCE']] 

In [ ]:
targets = bindf.P_HABITABLE

In [ ]:
final_features

### Number one rule of data science: know your data.

In [ ]:
final_features.shape

In [ ]:
final_features.describe()

#### There are some NaNs, e.g. shown by the "describe" property, which only counts numerical values.

### Counting missing data...

In [ ]:
for i in range(final_features.shape[1]):
    print(len(np.where(final_features.iloc[:,i].isna())[0]))

### ...and getting rid of them (Note: there are better imputing strategies!)

In [ ]:
final_features = final_features.dropna(axis = 0) #gets rid of any instance with at least one NaN in any column
final_features.shape

### Searching for outliers

Method 1 - plot!

In [ ]:
plt.hist(final_features.iloc[:,0], bins = 20, alpha = 0.5);

There is a remarkable outlier; The same happens for other features. But we could have also known from the difference between mean and median (which, in fact, is even more pronounced for orbital distance and period).

In [ ]:
final_features.describe()

In [ ]:
final_features = final_features[(np.abs(stats.zscore(final_features)) < 5).all(axis=1)] 

#This eliminates > 5 sigma outliers; however it counts from the mean so it might not be ideal

In [ ]:
targets = targets[final_features.index]

### Now reset index.

In [ ]:
final_features = final_features.reset_index(drop=True)

In [ ]:
final_features

### And don't forget to do the same for the label vector.

In [ ]:
targets = targets.reset_index(drop=True)

In [ ]:
targets

### Comparing the shapes, we can see that .... outliers were eliminated.

In [ ]:
targets.shape

### Check balance of data set

In [ ]:
#Simple way: count 0/1s, get fraction of total

In [ ]:
np.sum(targets)/len(targets)

In [ ]:
np.bincount(targets) #this shows the distribution of the two classes

### We can also explore the data by class, to get a sense of how the two classes differ from one another. For this, we need to concatenate the feature//labels data frames so we group objects label.

In [ ]:
#This generates a "view", not a new data frame

pd.concat([final_features, targets], axis=1)

In [ ]:
#We can group by label and take a look at summary statistics

pd.concat([final_features, targets], axis=1).groupby('P_HABITABLE').describe(percentiles = [])

### Ok, this all for preliminary data exploration. Time to deploy!

In [ ]:
Xtrain, Xtest, ytrain, ytest = train_test_split(final_features,targets,random_state=2)

In [ ]:
Xtrain.shape, Xtest.shape

We can just take a look at the train/test sets.

In [ ]:
plt.figure(figsize=(10,6))
cmap = matplotlib.colors.LinearSegmentedColormap.from_list("", ['#20B2AA','#FF00FF'])
a = plt.scatter(Xtrain['S_MASS'], Xtrain['P_PERIOD'], marker = '*',\
            c = ytrain, s = 100, cmap=cmap, label = 'Train')

a.set_facecolor('none')

a = plt.scatter(Xtest['S_MASS'], Xtest['P_PERIOD'], marker = 'o',\
            c = ytest, s = 100, cmap=cmap, label = 'Test')

plt.legend();

a.set_facecolor('none')

plt.yscale('log')
plt.xlabel('Mass of Parent Star (Solar Mass Units)')
plt.ylabel('Period of Orbit (days)');

bluepatch = mpatches.Patch(color='#20B2AA', label='Not Habitable')
magentapatch = mpatches.Patch(color='#FF00FF', label='Habitable')



ax = plt.gca()
leg = ax.get_legend()
leg.legendHandles[0].set_color('k')
leg.legendHandles[1].set_color('k')


plt.legend(handles=[leg.legendHandles[0],leg.legendHandles[1], magentapatch, bluepatch],\
           loc = 'lower right', fontsize = 14)

#plt.savefig('LargeHPTrainTest.png',dpi=300)

### Questions: 

- Based on this graph, would you expect DT or kNN to perform better? Why?

<br> 
- What kind of performance can we expect (qualitatively, is the information sufficient?) Do you expect to have latent (hidden) variables that might affect the outcome beyond those that we have?



In [ ]:
model = DecisionTreeClassifier(random_state=3)
model.fit(Xtrain,ytrain)

#### Let's visualize the graph!

In [ ]:
# Reminder: The features are always randomly permuted at each split. 
# Therefore, the best found split may vary, even with the same training data 
# and max_features=n_features, if the improvement of the criterion is identical 
# for several splits enumerated during the search of the best split. 
# To obtain a deterministic behaviour during fitting, random_state has to be fixed.

dot_data = StringIO()
export_graphviz(
            model,
            out_file =  dot_data,
            feature_names = ['Stellar Mass (M*)', 'Orbital Period (d)', 'Distance (AU)'],
            class_names = ['Not Habitable','Habitable'],
            filled = True,
rounded = True)
graph = pydotplus.graph_from_dot_data(dot_data.getvalue())  
nodes = graph.get_node_list()

for node in nodes:
    if node.get_label():
        values = [int(ii) for ii in node.get_label().split('value = [')[1].split(']')[0].split(',')]
        values = [255 * v / sum(values) for v in values]
        
        values = [int(255 * v / sum(values)) for v in values]
            
        if values[0] > values[1]:
            alpha = int(values[0] - values[1])
            alpha = '{:02x}'.format(alpha) #turn into hexadecimal
            color = '#20 B2 AA'+str(alpha)
        else:
            alpha = int(values[1] - values[0])
            alpha = '{:02x}'.format(alpha)
            color = '#FF 00 FF'+str(alpha)
        node.set_fillcolor(color)

#graph.write_png('Graph.png',dpi = 300)
        
Image(graph.create_png())

### Question: Can you predict the accuracy score on the train set?

### Let's take a look at train/test scores.

In [ ]:
print(metrics.accuracy_score(ytrain, model.predict(Xtrain)))
print(metrics.accuracy_score(ytest,model.predict(Xtest)))

This looks pretty high, but how does it compare with the accuracy of a lazy classifier that places everything in the "not habitable" category?

In [ ]:
#Dummy classifier

print(metrics.accuracy_score(ytest,np.zeros(len(ytest))))

### We can look at other metrics.

In [ ]:
print(metrics.precision_score(ytest,model.predict(Xtest)))

In [ ]:
print(metrics.recall_score(ytest,model.predict(Xtest)))

Not perfect, but not horrible.

In [ ]:
np.sum(model.predict(Xtest)), np.sum(ytest)

In [ ]:
print(metrics.confusion_matrix(ytest,model.predict(Xtest)))

### You know what we would need in order to understand exactly how the model is working? A confusion matrix!

In [ ]:
def plot_confusion_matrix(cm, classes,
                          normalize=False,
                          title='Confusion matrix',
                          cmap=plt.cm.Blues):
    """
    This function prints and plots the confusion matrix.
    Normalization can be applied by setting `normalize=True`.
    """
    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
        print("Normalized confusion matrix")
    else:
        print('Confusion matrix, without normalization')

    plt.figure(figsize=(7,6))
    print(cm)
    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=45)
    plt.yticks(tick_marks, classes)

    fmt = '.2f' if normalize else 'd'
    thresh = cm.max() / 2.
    for i in range(cm.shape[0]):
        for j in range(cm.shape[1]):
            plt.text(j, i, format(cm[i, j], fmt),
                 horizontalalignment="center", verticalalignment="center",
                 color="green" if i == j else "red", fontsize = 30)

    plt.tight_layout()
    plt.ylabel('True label')
    plt.xlabel('Predicted label')

### We can plot the confusion matrix (note that so far, we have only looked at one test fold)

In [ ]:
cm = metrics.confusion_matrix(ytest,model.predict(Xtest))
plot_confusion_matrix(cm, ['Not Hab','Hab'], cmap = plt.cm.Pastel2)
#plt.savefig('CM.png', dpi = 300)

### Three flavors of k-fold Cross Validation.

Note: you can fix the random seed for exactly reproducible behavior.

In [ ]:
# This is the standard version. Important: it doesn't shuffle the data, 
# so if your positive examples are all at the beginning or all the end, it might lead to disastrous results.

cv1 = KFold(n_splits = 5)

#This is v2: shuffling added (recommended!)

cv2 = KFold(shuffle = True, n_splits = 5, random_state=5)

# STRATIFICATION ensures that the class distributions in each split resembles those of the 
# entire data set (mightscores1['test_score'].mean(), scores1['test_score'].std()scores1['test_score'].mean(), scores1['test_score'].std()### Effect of stratification: let's look at the class count in each set of splits.

cv3 = StratifiedKFold(shuffle = True, n_splits = 5, random_state=5)

### Effect of stratification: let's look at the class count in each set of splits.

In [ ]:
for train, test in cv1.split(final_features, targets): #Just how they are in original data set
...     print('train -  {}   |   test -  {}'.format(
...         np.bincount(targets.loc[train]), np.bincount(targets.loc[test])))

In [ ]:
for train, test in cv2.split(final_features, targets): #One random selection
...     print('train -  {}   |   test -  {}'.format(
...         np.bincount(targets.loc[train]), np.bincount(targets.loc[test])))

In [ ]:
for train, test in cv3.split(final_features, targets): #One adjusted-for random selection
...     print('train -  {}   |   test -  {}'.format(
...         np.bincount(targets.loc[train]), np.bincount(targets.loc[test])))

#### The handy function cross\_validate provides the scores (specified by the chosen scoring parameter), in dictionary form.

In [ ]:
scores1 = cross_validate(DecisionTreeClassifier(), final_features, targets, cv = cv1, scoring = 'accuracy')

scores2 = cross_validate(DecisionTreeClassifier(), final_features, targets, cv = cv2, scoring = 'accuracy')

scores3 = cross_validate(DecisionTreeClassifier(), final_features, targets, cv = cv3, scoring = 'accuracy')

In [ ]:
scores1

#### We can now calculate an average and standard deviation.

In [ ]:
scores1['test_score'].mean(), scores1['test_score'].std()

In [ ]:
scores2['test_score'].mean(), scores2['test_score'].std()

In [ ]:
scores3['test_score'].mean(), scores3['test_score'].std()

#### Question: are the differences statistically significant?

### Let's now use recall as our scoring parameter. Will the model change?

In [ ]:
scores1 = cross_validate(DecisionTreeClassifier(), final_features, targets, cv = cv1, scoring = 'recall')

scores2 = cross_validate(DecisionTreeClassifier(), final_features, targets, cv = cv2, scoring = 'recall')

scores3 = cross_validate(DecisionTreeClassifier(), final_features, targets, cv = cv3, scoring = 'recall')

In [ ]:
print(scores1['test_score'].mean(), scores1['test_score'].std())
print(scores2['test_score'].mean(), scores2['test_score'].std())
print(scores3['test_score'].mean(), scores3['test_score'].std())

### If desired, I can ask for the train scores as well. This is very helpful when diagnosing bias vs variance.

In [ ]:
scores1 = cross_validate(DecisionTreeClassifier(), final_features, targets, cv = cv1, scoring = 'recall', \
                         return_train_score = True)

scores2 = cross_validate(DecisionTreeClassifier(), final_features, targets, cv = cv2, scoring = 'recall', \
                         return_train_score = True)

scores3 = cross_validate(DecisionTreeClassifier(), final_features, targets, cv = cv3, scoring = 'recall', 
                         return_train_score = True)

In [ ]:
print(scores1['test_score'].mean(), scores1['train_score'].mean())
print(scores2['test_score'].mean(), scores2['train_score'].mean())
print(scores3['test_score'].mean(), scores3['train_score'].mean())

### The cross\_validate function is useful to calculate the score, but does not produce predicted labels.

#### These can be obtained by using the cross\_val\_predict function, which saves the predictions for each of the k test folds, and compiles them together.

In [ ]:
model1 = DecisionTreeClassifier(random_state=3)
scores1 = cross_val_score(model1, final_features, targets, cv = cv1, scoring = 'accuracy')
y1 = cross_val_predict(model1, final_features, targets, cv = cv1)

In [ ]:
y1

In [ ]:
np.sum(y1) #trick to see how many planets are predicted to be habitable (predicted label = 1)

In case you don't believe that the model is the same, let's change the scoring parameter and train it again:

In [ ]:
model2 = DecisionTreeClassifier(random_state=3)
scores2 = cross_val_score(model2, final_features, targets, cv = cv1, scoring = 'recall')
y2 = cross_val_predict(model2, final_features, targets, cv = cv1)

In [ ]:
np.sum(y2)

In [ ]:
np.sum(y1-y2)

In [ ]:
metrics.confusion_matrix(targets,y1)

In [ ]:
metrics.confusion_matrix(targets,y2)

However, things may change if I use a different cross validation scheme:

In [ ]:
model1 = DecisionTreeClassifier(random_state=3)
scores1 = cross_val_score(model1, final_features, targets, cv = cv1, scoring = 'accuracy')
y1 = cross_val_predict(model1, final_features, targets, cv = cv1)

In [ ]:
model2 = DecisionTreeClassifier(random_state=3)
scores2 = cross_val_score(model2, final_features, targets, cv = cv2, scoring = 'accuracy')
y2 = cross_val_predict(model2, final_features, targets, cv = cv2)

In [ ]:
np.sum(y1-y2)

In [ ]:
np.sum(y1)

In [ ]:
metrics.confusion_matrix(targets,y1)

In [ ]:
metrics.confusion_matrix(targets,y2)

In [ ]:
model3 = DecisionTreeClassifier(random_state=3)
scores3 = cross_val_score(model3, final_features, targets, cv = cv3, scoring = 'accuracy')
y3 = cross_val_predict(model3, final_features, targets, cv = cv3)

In [ ]:
metrics.confusion_matrix(targets,y3)

This is a good reminder that the CM is also only one possible realization of the model, and is subject to random fluctuations just like the cross validation scores.

## Now switching to kNN classifier. Oh wait, that's for homework! :) 